Here’s a simple example program demonstrating **Retrieval-Augmented Generation (RAG)** using Python, with tools like Hugging Face’s libraries.

For this example, we’ll use a pre-trained retriever model like **DPR (Dense Passage Retriever)** for retrieval and T5 for generation.

In [ ]:
pip install transformers datasets faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, DPRContextEncoder, DPRContextEncoderTokenizer, DPRQuestionEncoder, DPRQuestionEncoderTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [ ]:
# Sample knowledge base
knowledge_base = [
    "The Eiffel Tower is located in Paris, France.",
    "Python is a popular programming language used for AI and machine learning.",
    "The human heart has four chambers: two atria and two ventricles.",
    "Albert Einstein developed the theory of relativity.",
    "RAG stands for Retrieval-Augmented Generation in AI.",
    "Damon Salvatore, played by Ian Somerhalder, is a main character in The Vampire Diaries.",
    "He's a centuries-old vampire and the older brother of Stefan Salvatore.",
    "Damon is initially portrayed as the show's villain but later becomes a complex and multidimensional character.",
    "His love for Elena Gilbert, played by Nina Dobrev, is a central theme throughout the series.",
    "Damon's charming, manipulative, and sometimes ruthless nature makes him a compeling and intriguing character.",
    "Stefan Salvatore, played by Paul Wesley, is a main character in The Vampire Diaries. He's a centuries-old vampire and the younger brother of Damon Salvatore. Stefan is initially portrayed as the show's hero but later becomes a complex and multidimensional character. His love for Elena Gilbert is a central theme throughout the series. Stefan's compassionate, responsible, and sometimes brooding nature makes him a relatable and endearing character."


]

In [ ]:
# Step 1: Encode the Knowledge Base with DPR Context Encoder
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
context_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

encoded_knowledge_base = context_encoder(**context_tokenizer(knowledge_base, return_tensors="pt", padding=True, truncation=True)).pooler_output


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification mod

In [ ]:
# Step 2: Query Encoding with DPR Question Encoder
query = input("Enter your query: ")
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
question_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

query_embedding = question_encoder(**question_tokenizer(query, return_tensors="pt", truncation=True)).pooler_output

Enter your query: who is Damon's brother


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
# Step 3: Find the Most Relevant Context Using Cosine Similarity
similarities = cosine_similarity(query_embedding.detach().numpy(), encoded_knowledge_base.detach().numpy())
most_relevant_index = np.argmax(similarities)

retrieved_context = knowledge_base[most_relevant_index]
print(f"Retrieved Context: {retrieved_context}")


Retrieved Context: Stefan Salvatore, played by Paul Wesley, is a main character in The Vampire Diaries. He's a centuries-old vampire and the younger brother of Damon Salvatore. Stefan is initially portrayed as the show's hero but later becomes a complex and multidimensional character. His love for Elena Gilbert is a central theme throughout the series. Stefan's compassionate, responsible, and sometimes brooding nature makes him a relatable and endearing character.


In [ ]:
# Step 4: Use T5 for Generation
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-small")

In [ ]:

# Combine query and retrieved context
input_text = f"question: {query} context: {retrieved_context}"
input_ids = t5_tokenizer(input_text, return_tensors="pt").input_ids

In [ ]:

# Generate the final answer
output_ids = t5_model.generate(input_ids, max_length=50, num_beams=2)
generated_answer = t5_tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(f"Generated Answer: {generated_answer}")

Generated Answer: Stefan Salvatore
